## Load Packages and Data

In [ ]:
from stock_flow import *
import time
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

def add_used_vehicle_value(model):
    used_prices = pd.read_excel("Used HDV Prices.xlsx")
    model["used_value"] = 0
    for v in model.veh_type.unique():
        for a in model.loc[model.veh_type==v,"age"].unique():
            model.loc[(model.veh_type==v) & (model.age == a), "used_value"] = (used_prices.loc[used_prices.Age==a,v].item()*
                                                                       model.loc[(model.veh_type==v) & (model.age == a), "early_ret"])
    return model

In [ ]:
## Get EMFAC stock profiles:
vmt_prof = pd.DataFrame(index = np.arange(2019,2047), columns = ["T7", "T6_OOS","T7_OOS", "Buses","MH","MC","LHD1","LHD2","T7_Port","T6"])
for v in ["T7", "T6_OOS","T7_OOS", "Buses","MH","MC","LHD1","LHD2","T7_Port","T6"]:
    t = pd.read_csv(f"EMFAC_BAU/EMFAC_bau_model_{v}.csv")
    t_vmt = t.groupby("year").agg({"vmt":"sum"})
    vmt_prof[v] = t_vmt

In [ ]:
stock_prof = pd.DataFrame(index = np.arange(2019,2047), columns = ["T7", "T6_OOS","T7_OOS", "Buses","MH","MC","LHD1","LHD2","T7_Port","T6"])
for v in ["T7", "T6_OOS","T7_OOS", "Buses","MH","MC","LHD1","LHD2","T7_Port","T6"]:
    t = pd.read_csv(f"EMFAC_BAU/EMFAC_bau_model_{v}.csv")
    t_stock = t.groupby("year").agg({"stock":"sum"})
    stock_prof[v] = t_stock

In [ ]:
vmt_prof.to_csv("emfac_vmt_prof.csv")
stock_prof.to_csv("emfac_stock_prof.csv")

# Model Runs

## BAU Scenario

In [ ]:
start_year = 2019
end_year = 2045
veh_types = ["T7", "T6_OOS","T7_OOS", "Buses","MH","MC","LHD1","LHD2","T7_Port","T6"]
fuel_types = ["Diesel","Gasoline","Natural Gas","Electricity","Hydrogen","Hybrid"]
input_file = "Inputs_new_surv_my2020.xlsx"#"Inputs_adj.xlsx"
vmt_mode = "none"

model_BAU = stock_and_flow_baseline(start_year, end_year, veh_types, fuel_types, input_file, vmt_mode)

In [ ]:
bau_stock = extract_stock_profile(model_BAU, ["T6","T7","T6_OOS","T7_OOS","T7_Port","Buses","MH","MC","LHD1","LHD2"])
bau_stock.to_csv("bau_stock_prof_new.csv")

bau_vmt = extract_vmt_profile(model_BAU, ["T6","T7","T6_OOS","T7_OOS","T7_Port","Buses","MH","MC","LHD1","LHD2"])
bau_vmt.to_csv("bau_vmt_prof_new.csv")

In [ ]:
start = time.time()
start_year = 2019
end_year = 2045
veh_types = ["T6","T7","T6_OOS","T7_OOS", "Buses","MC","LHD1","LHD2","T7_Port","MH"]
fuel_types = ["Diesel","Gasoline","Natural Gas","Electricity","Hydrogen","Hybrid"]

input_file = "Inputs_new_surv_my2020.xlsx"
ret_ages = [5]
ret_years = [2100]
vmt_mode = "prof"
stock_prof = True

frac_elec = {}
frac_elec["T6"] = 1
frac_elec["T6_OOS"] = 1
frac_elec["T7"] = 1
frac_elec["T7_OOS"] = 1
frac_elec["T7_Port"] = 1
frac_elec["LHD1"] = 1
frac_elec["LHD2"] = 1
frac_elec["MC"] = 1
frac_elec["MH"] = 1
frac_elec["Buses"] = 1


ret_mode = ["uniform"]
sales_mode = ""
replacement_mode = "ZEV_only"



retirement_profile = {}
retirement_mode = {}

for v in veh_types:
    for ra in ret_ages:
        for ry in ret_years:
            retirement_profile[v] = {}
            retirement_profile[v]["ret_age"] = ra
            retirement_profile[v]["ret_year"] = ry
            retirement_profile[v]["phase_in"] = 0
            retirement_mode[v] = ret_mode
            for zev_year in [2100]:
                target_year = {}
                target_year[v] = zev_year
                model = stock_and_flow_sales_ret(start_year, end_year, [v], fuel_types, input_file, stock_prof, vmt_mode, 
                                                 target_year, frac_elec, [retirement_profile], retirement_mode, sales_mode, replacement_mode)
                add_used_vehicle_value(model)
                model.to_csv(f"Paper_results_new/model_{v}_BAU.csv")
end = time.time()
print(start, end)

## ZEV Mandate Scenarios

In [ ]:
#Sales only scenarios
start = time.time()
start_year = 2019
end_year = 2045
veh_types = ["T6","T7","T6_OOS","T7_OOS", "Buses","MC","LHD1","LHD2","T7_Port","MH"]
fuel_types = ["Diesel","Gasoline","Natural Gas","Electricity","Hydrogen","Hybrid"]

input_file = "Inputs_new_surv_my2020.xlsx"
ret_ages = [5]
ret_years = [2100]
vmt_mode = "prof"
stock_prof = True

frac_elec = {}
frac_elec["T6"] = 1
frac_elec["T6_OOS"] = 1
frac_elec["T7"] = 1
frac_elec["T7_OOS"] = 1
frac_elec["T7_Port"] = 1
frac_elec["LHD1"] = 1
frac_elec["LHD2"] = 1
frac_elec["MC"] = 1
frac_elec["MH"] = 1
frac_elec["Buses"] = 1

ret_mode = ["uniform"]
sales_mode = "linear"
replacement_mode = "ZEV_only"



retirement_profile = {}
retirement_mode = {}

for v in veh_types:
    for ra in ret_ages:
        for ry in ret_years:
            retirement_profile[v] = {}
            retirement_profile[v]["ret_age"] = ra
            retirement_profile[v]["ret_year"] = ry
            retirement_profile[v]["phase_in"] = 0
            retirement_mode[v] = ret_mode
            for zev_year in [2025, 2030, 2035, 2040, 2045]:
                target_year = {}
                target_year[v] = zev_year
                model = stock_and_flow_sales_ret(start_year, end_year, [v], fuel_types, input_file, stock_prof, vmt_mode, 
                                                 target_year, frac_elec, [retirement_profile], retirement_mode, sales_mode, replacement_mode)
                add_used_vehicle_value(model)
                model.to_csv(f"Paper_results_new/model_{v}_{zev_year}.csv")
end = time.time()
print(start, end)

## Early Retirement Scenarios

In [ ]:
#Sales and retirement only scenarios
start = time.time()
start_year = 2019
end_year = 2045
veh_types = ["T6","T7","T6_OOS","T7_OOS", "Buses","MC","LHD1","LHD2","T7_Port","MH"]#["T6","T7", #T7 didn't finish
fuel_types = ["Diesel","Gasoline","Natural Gas","Electricity","Hydrogen","Hybrid"]
#input_file = "Inputs_mixed_scenario_no_early_ret_EMFAC_sales<1_2020_surv_updatedfe_vint_vmt_actual_surv_2.xlsx"
input_file = "Inputs_new_surv_my2020.xlsx"
ret_ages = [5,10, 15, 20, 25]
ret_years = [2025, 2030, 2035,2040, 2045]
vmt_mode = "prof"
stock_prof = True

frac_elec = {}
frac_elec["T6"] = 1
frac_elec["T6_OOS"] = 1
frac_elec["T7"] = 1
frac_elec["T7_OOS"] = 1
frac_elec["T7_Port"] = 1
frac_elec["LHD1"] = 1
frac_elec["LHD2"] = 1
frac_elec["MC"] = 1
frac_elec["MH"] = 1
frac_elec["Buses"] = 1

# ret_profile= {}
# ret_profile["ret_age"] = 30
# ret_profile["ret_year"] = 2100
# ret_profile["phase_in"] = 0
ret_mode = ["uniform"]
sales_mode = "linear"
replacement_mode = "ZEV_only"



retirement_profile = {}
retirement_mode = {}
# retirement_profile["T6"] = {}
# retirement_profile["T6"]["phase_in"] = 5
for v in veh_types:
    for ra in ret_ages:
        for ry in ret_years:
            retirement_profile[v] = {}
            retirement_profile[v]["ret_age"] = ra
            retirement_profile[v]["ret_year"] = ry
            retirement_profile[v]["phase_in"] = 0
            retirement_mode[v] = ret_mode
            for zev_year in [2035,2040, 2045]:
                target_year = {}
                target_year[v] = zev_year
                model = stock_and_flow_sales_ret(start_year, end_year, [v], fuel_types, input_file, stock_prof, vmt_mode, 
                                                 target_year, frac_elec, [retirement_profile], retirement_mode, sales_mode, replacement_mode)
                model.to_csv(f"Paper_results_new/model_{v}_{zev_year}_{ra}_{ry}.csv")
end = time.time()
print(start, end)